# Powering Through

**Name(s)**: Andrea González Martín & Andrés Riera Ortiz

**Website Link**: https://andiigonzalez.github.io/Power_Outage_Analysis/index.html

In [40]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'
import matplotlib as plt

from dsc80_utils import * 

## Step 1: Introduction

In [41]:
# Open the csv file and drop columns that are not needed for the analysis
#pd.set_option('display.max_columns', None) # display all columns of the dataframe 
outages = pd.read_csv('/Users/andigonzalez/Desktop/DSC80/Power_Outage_Analysis/outages.csv') # read the csv file
outages.drop(['POSTAL.CODE', 'HURRICANE.NAMES', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'RES.PRICE','COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE','RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN','COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUSTOMERS', 'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPPCT_UC', 'POPDEN_UC', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT', 'PCT_WATER_INLAND', 'AREAPCT_URBAN','POPPCT_URBAN' ], axis=1, inplace=True)



In [42]:
# Save the top rows of dataframe head an html file to include in the webpage
html_file = "/Users/andigonzalez/Desktop/DSC80/Power_Outage_Analysis/assets/images/outages_head.html"
outages.head(10).to_html(html_file, index=False)


## Step 2: Data Cleaning and Exploratory Data Analysis

### Data Cleaning

In [43]:
# Clean categorical values of leading or trailing spaces 
outages['U.S._STATE'] = outages['U.S._STATE'].str.strip()
outages['NERC.REGION'] = outages['NERC.REGION'].str.strip()
outages['CLIMATE.REGION'] = outages['CLIMATE.REGION'].str.strip()
outages['CLIMATE.CATEGORY'] = outages['CLIMATE.CATEGORY'].str.strip()
outages['CAUSE.CATEGORY'] = outages['CAUSE.CATEGORY'].str.strip()
outages['CAUSE.CATEGORY.DETAIL'] = outages['CAUSE.CATEGORY.DETAIL'].str.strip()



In [44]:
# combine dates and times into a single column.
outages['OUTAGE.START']  = outages['OUTAGE.START.DATE'] + " "+ outages['OUTAGE.START.TIME']
# reformat variables to datetime format
outages['OUTAGE.START'] = pd.to_datetime(outages['OUTAGE.START'], format='%Y-%m-%d %H:%M:%S')


In [97]:
# Replace/Substitute values of 0 in columns OUTAGE.DURATION, DEMAND.LOSS.MW, and CUSTOMERS.AFFECTED
outages['OUTAGE.DURATION'] = outages['OUTAGE.DURATION'].replace(0, np.nan)
outages['DEMAND.LOSS.MW'] = outages['DEMAND.LOSS.MW'].replace(0, np.nan)
outages['CUSTOMERS.AFFECTED'] = outages['CUSTOMERS.AFFECTED'].replace(0, np.nan)
outages['CAUSE.CATEGORY'].unique()
outages.columns


Index(['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION',
       'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'OUTAGE.START.DATE',
       'OUTAGE.START.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.SALES',
       'COM.SALES', 'IND.SALES', 'TOTAL.SALES', 'POPULATION', 'POPDEN_URBAN',
       'POPDEN_RURAL', 'OUTAGE.START'],
      dtype='object')

### Univariate Analysis

In [46]:
import matplotlib.pyplot as plt
import plotly.express as px 
# Group data by year to produce a univariate analysis of the number of outages per year. Create a bar plot with the corresponding data.
outages_by_year = outages.groupby('YEAR').count()['OBS']
outages_by_year_df = outages_by_year.reset_index()
fig = px.bar(outages_by_year_df, x='YEAR', y='OBS', title='Number of Power Outages by Year', labels={'YEAR': 'Year', 'OBS': 'Number of Outages'}, color_discrete_sequence=['orange'])

# Customize layout
fig.update_layout(
    title={
        'text': 'Number of Power Outages by Year',
        'x': 0.5,  # Center the title
        'y': 0.98,  # Add padding above the plot
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')  # Custom font for title
    },
    xaxis_title_font=dict(size=14),  # Font size for x-axis title
    yaxis_title_font=dict(size=14),  # Font size for y-axis title
    xaxis=dict(tickmode='linear'),  # Ensure all years are displayed
    width=700,  # Make the plot wider
    height=300   # Adjust height if needed
)

# Show the plot
fig.show()
pio.write_html(fig, file='assets/images/outages_by_year.html', auto_open=False)


In [47]:
import pandas as pd
import matplotlib.pyplot as plt

# Example DataFrame (replace this with your actual data)
# outages = pd.read_csv('outages_data.csv')

outages2 = outages.dropna(subset=['CAUSE.CATEGORY'])
# Group data by cause and count the number of occurrences
cause_counts = outages2.groupby('CAUSE.CATEGORY').size()

# Create interactive pie chart with Plotly
fig = go.Figure(data=[go.Pie(
    labels=cause_counts.index.str.capitalize(), 
    values=cause_counts.values,
    textinfo='percent+label',
    textposition='inside',
    textfont=dict(size=14, family='Arial, Helvetica, sans-serif'),
    hoverinfo='label+percent')])

# Customize layout with explicit sans-serif font
fig.update_layout(
    title={
        'text': 'Distribution of Power Outages by Cause from 2000 to 2016',
        'font': {'size': 20, 'family': 'Arial, Helvetica, sans-serif' } },
        font=dict(family='Arial, Helvetica, sans-serif'))
fig.show()
# Save as interactive HTML file
fig.write_html('assets/images/distribution_of_power_outage_cause.html', auto_open=False)

In [48]:
# Group data by state variable to produce a univariate analysis of the number of total outages per state

outages_by_state = outages.groupby('U.S._STATE').count()['OBS'].reset_index()
outages_by_state_plot = px.line(outages_by_state, x='U.S._STATE', y='OBS', title='Number of Power Outages by State from 2000 to 2016', labels={'U.S._STATE': 'State', 'OBS': 'Number of Outages'}, color_discrete_sequence=['skyblue'])

outages_by_state_plot.update_layout(
    title={
        'text': 'Number of Power Outages by State from 2000 to 2016',
        'x': 0.5,  # Center the title
        'y': 0.98,  # Add padding above the plot
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')  # Custom font for title
    },
    xaxis_title_font=dict(size=14),  # Font size for x-axis title
    yaxis_title_font=dict(size=14),  # Font size for y-axis title
    xaxis=dict(tickmode='linear'),  # Ensure all years are displayed
    yaxis=dict(
        tickmode='array',  # Specify custom tick values
        tickvals=list(range(0, outages_by_state['OBS'].max() + 50, 50)),  # Ticks at 50 increments
        title='Number of Outages'
    ),
    width=800,  # Make the plot wider
    height=400   # Adjust height if needed
)

# Show the plot
outages_by_state_plot.show()
pio.write_html(outages_by_state_plot, file='assets/images/outages_by_state.html', auto_open=False)


### Bivariate Analysis

#### Bivariate Analysis #1: Interactive Map of Power Outages Experienced By Climate Regions


In [49]:
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go

# Create a dictionary mapping regions to their states
region_states = {
    'Northeast': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA'],
    'Southeast': ['FL', 'GA', 'AL', 'NC', 'SC', 'VA'],
    'Central': ['IL', 'MO', 'IN', 'KY', 'WV', 'OH', 'TN'],
    'Southwest': ['AZ', 'CO', 'UT', 'NM'],
    'West': ['NV', 'CA'],
    'Northwest': ['OR', 'WA', 'ID'],
    'South': ['KS', 'TX', 'OK', 'LA', 'AR', 'MS'],
    'West North Central': ['ID', 'MT', 'NE', 'ND', 'SD', 'WY'],
    'East North Central': ['MN', 'IA', 'WI', 'MI']
}

# Outages data
outages_data = {
    'Region': ['Central', 'East North Central', 'Northeast', 'Northwest', 
               'South', 'Southeast', 'Southwest', 'West', 
               'West North Central'],
    'Outages': [200, 138, 350, 132, 229, 153, 92, 217, 17]
}

# Create DataFrame
flattened_data = [(region, state) for region, states in region_states.items() for state in states]
df_regions = pd.DataFrame(flattened_data, columns=['Region', 'State'])

# Convert outages_data to a DataFrame
df_outages = pd.DataFrame(outages_data)

# Merge the regions DataFrame with the outages data
merged_df = df_regions.merge(df_outages, on='Region', how='left')
region_centers = {
    'Northeast': {'lat': 42.0, 'lon': -72.0},
    'Southeast': {'lat': 33.0, 'lon': -84.0},
    'Central': {'lat': 39.0, 'lon': -88.0},
    'Southwest': {'lat': 36.0, 'lon': -108.0},
    'West': {'lat': 37.0, 'lon': -119.0},
    'Northwest': {'lat': 45.0, 'lon': -120.0},
    'South': {'lat': 33.0, 'lon': -97.0},
    'West North Central': {'lat': 45.0, 'lon': -105.0},
    'East North Central': {'lat': 43.0, 'lon': -89.0}
}

# Create a base choropleth map
fig = px.choropleth(
    merged_df,
    locations="State",                # The column with state abbreviations
    locationmode="USA-states",        # Map mode for US states
    color="Outages",                  # Color by the total outages in the region
    hover_name="Region",              # Display the region name on hover
    title="Regional Outages in the USA",
    scope="usa",                      # Limit map to USA
    color_continuous_scale=px.colors.sequential.Plasma
)

# Add annotations for region names


# Update layout to ensure sans-serif font for the plot text
fig.update_layout(
    font=dict(family="sans-serif")  # Set the font family for the whole plot to sans-serif
)

# Show the map
fig.show()
pio.write_html(fig, file='assets/images/outages_by_region_map.html', auto_open=False)

#### Bivariate Analysis #2: Stacked Bar Plot of Quantifying the Number of Power Outages and the Cause by Climate Region


In [50]:
import plotly.io as pio

# Create the grouped DataFrame for the plot
a = outages.groupby(['CLIMATE.REGION', 'CAUSE.CATEGORY']).count()['OBS']
df = a.reset_index()
df['CAUSE.CATEGORY'] = df['CAUSE.CATEGORY'].str.capitalize()
stacked_plot = px.bar(
    df,
    x='CLIMATE.REGION',
    y='OBS',
    color='CAUSE.CATEGORY'
)

# Update the layout of the plot
stacked_plot.update_layout(
    font=dict(family='Sans-serif'),
    title={
        'text': 'Number of Outages and their Cause Category per Region',
        'x': 0.4,  # Center the title horizontally
        'y': 0.98,  # Adjust vertical positioning of the title
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')
    },
    xaxis_title='Climate Region',  # Title for the x-axis
    yaxis_title='Number of Outages',  # Title for the y-axis
    xaxis=dict(
        title_font=dict(size=14),  # Font size for x-axis title
        tickmode='linear'
    ),
    yaxis=dict(
        title_font=dict(size=14)  # Font size for y-axis title
    ),
    legend_title_text='Cause Category',  # Legend title
    legend=dict(
        title_font=dict(size=14),  # Font size for legend title
        font=dict(size=12)  # Font size for legend items
    ),
    width=800,  # Adjust the width of the plot
    height=400   # Adjust the height of the plot
)

# Show the plot
stacked_plot.show()

# Save the plot as an HTML file
pio.write_html(stacked_plot, file='assets/images/bivariate_stacked_barplot.html', auto_open=False)


#### Bivariate Analysis #3: Stacked Bar Plot of Quantifying the Number of Power Outages and the Cause by State


In [51]:
import plotly.io as pio

# Create the grouped DataFrame for the plot
outage_causes_by_State = outages.groupby(['U.S._STATE', 'CAUSE.CATEGORY']).count()['OBS']
state_causes= outage_causes_by_State.reset_index()
state_causes['CAUSE.CATEGORY'] = state_causes['CAUSE.CATEGORY'].str.capitalize()
stacked_plot_state_cause = px.bar(
    state_causes,
    x='U.S._STATE',
    y='OBS',
    color='CAUSE.CATEGORY'
)

# Update the layout of the plot
stacked_plot_state_cause.update_layout(
    font=dict(family='Sans-serif'),
    title={
        'text': 'Number of Outages and their Cause Category per State',
        'x': 0.4,  # Center the title horizontally
        'y': 0.98,  # Adjust vertical positioning of the title
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')
    },
    xaxis_title='State',  # Title for the x-axis
    yaxis_title='Number of Outages',  # Title for the y-axis
    xaxis=dict(
        title_font=dict(size=14),  # Font size for x-axis title
        tickmode='linear'
    ),
    yaxis=dict(
        title_font=dict(size=14)  # Font size for y-axis title
    ),
    legend_title_text='Cause Category',  # Legend title
    legend=dict(
        title_font=dict(size=14),  # Font size for legend title
        font=dict(size=12)  # Font size for legend items
    ),
    width=800,  # Adjust the width of the plot
    height=400   # Adjust the height of the plot
)

# Show the plot
stacked_plot_state_cause.show()

# Save the plot as an HTML file
pio.write_html(stacked_plot_state_cause, file='assets/images/bivariate_stacked_barplot_by_state.html', auto_open=False)


### Aggregate Variables

#### Pivot Table #1:
1. Grouped data by `CLIMATE.REGION`, `YEAR`, and `CAUSE.CATEGORY` to see how each of the climate region's vulnerability to specific causes and the number of outages they have experienced per cause per year. 

2. Created a Pivot Table with `pd.pivot_table()` using the dataframe of grouped values we created. We also replaced missing values with 0 for readability. 

3. Added a column called `Total Outages` to calcualte the total outages per climate region per year


In [52]:
# Group data by climate region, Year and Cause Category of the Power outage. Then create a pivot table so that we can understand if there are any trends within these variables. 
b = outages.groupby(['YEAR', 'CLIMATE.REGION', 'CAUSE.CATEGORY']).count()['OBS'] # group the data by the relevant variables
df2 = b.reset_index() # reset index
df2['CAUSE.CATEGORY'] = df2['CAUSE.CATEGORY'].str.capitalize() # Capitalize the Cause categories for earier visualization and aesthetic

# Create a piuvot table
# For easier understanding, fill in missing values (NaN) as 0
pivot_table = pd.pivot_table(
    df2,
    values='OBS',
    index=['YEAR', 'CLIMATE.REGION'], # select the Multindex for the pivot table
    columns='CAUSE.CATEGORY',
    aggfunc='sum',  # Aggregation function to calculate number of outages with the same cause within each climate region and year
    fill_value=0    # Replace NaN with 0 for readability
)
# Add an extra column that computed the total jumber of outages per year per climate region
pivot_table['Total Outages'] = pivot_table.sum(axis=1)


pivot_table.to_html('assets/images/pivot_table_outages.html', border=1, col_space=100)  # Convert the pivot table to HTML and save it into my assets/images folder



#### Pivot Table #2:

Thispi vot table is very similar to our first one except we did not include year. 

1. Grouped data by `CLIMATE.REGION`, and `CAUSE.CATEGORY` to see how each of the climate region's vulnerability to specific causes and the number of outages they have experienced per cause. 

2. Created a Pivot Table with `pd.pivot_table()` using the dataframe of grouped values we created. We also replaced missing values with 0 for readability. Additionally, we used the aggregate function `sum` to obtain the number of power outages that occured due to a specific cause per Climate Region.

3. Added a column called `Total Outages` to calcualte the total outages per climate region. 

In [53]:
# Grouped the data by Climate Region and Cause Category
c = outages.groupby(['CLIMATE.REGION', 'CAUSE.CATEGORY']).count()['OBS']
df3 = c.reset_index()
df3['CAUSE.CATEGORY'] = df2['CAUSE.CATEGORY'].str.capitalize()
pivot_table2 = pd.pivot_table(
    df2,
    values='OBS',
    index=['CLIMATE.REGION'],
    columns='CAUSE.CATEGORY',
    aggfunc='sum',  # Aggregation function to calculate number of outages with the same cause within each climate region
    fill_value=0    # Replace NaN with 0 for readability
)
pivot_table2['Total Outages'] = pivot_table2.sum(axis=1) # Added a Total Column to calculate the total number of outages per Climate Region


pivot_table2.to_html('assets/images/pivot_table_outages_by_year_&_climate_region.html', border=1, col_space=100)  # Convert the pivot table to HTML and save it into my assets/images folder
pivot_table2

CAUSE.CATEGORY,Equipment failure,Fuel supply emergency,Intentional attack,Islanding,Public appeal,Severe weather,System operability disruption,Total Outages
CLIMATE.REGION,,,,,,,,
Central,7,4,38,3,2,135,11,200
East North Central,3,5,20,1,2,104,3,138
Northeast,5,14,135,1,4,176,15,350
...,...,...,...,...,...,...,...,...
Southwest,5,2,64,1,1,10,9,92
West,21,17,31,28,9,70,41,217
West North Central,1,1,4,5,2,4,0,17


#### Pivot Table #3: Total Number of Outages by Month Occurance and Cause
This pivot table is to understand the distribution of outages over the months in a year and also the relationship between certain outages causes and the month they occured in . 

1. Grouped data by `CAUSE.CATEGORY`, `CAUSE.CATEGORY.DETAIL`, and `MONTH` to see if each state's yearly consumption of electricty and number of outages have a trend or not. 

2. Created a Pivot Table with `pd.pivot_table()` using the dataframe of grouped values we created. We also replaced missing values with 0 for readability and used the aggregate function `sum` to compute the number of outages that occured in a specific month due to the same cause category and category detail.

3. Added a column called `Total Outages` to calcualte the total outages per climate region per year

In [54]:

a = outages.groupby(['CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'MONTH']).count()['OBS']
df = a.reset_index()

pivot_table3 = df.pivot_table(
    index=['CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL'],
    columns='MONTH',
    values='OBS',
    fill_value=0,  # Fill missing values with 0
    aggfunc='sum'  # Sum if there are multiple entries per group
)
pivot_table3 = pivot_table3.loc[(pivot_table3 != 0).any(axis=1)] # Deleted any rows that were all 0's for visual interpretation

pivot_table3.columns = pd.to_datetime(pivot_table3.columns, format='%m').strftime('%b') # Converted numerical values of months into abbreviations for the month. ie. 1.0 = 'Jan'
# Convert the pivot table to HTML and save it
pivot_table3['Total'] = pivot_table3.sum(axis=1)

# Add a 'Total' row for the total outages per month (by summing columns)
#total_row = pivot_table3.sum(axis=0).to_frame().T
#pivot_table3 = pd.concat([pivot_table3, total_row])

pivot_table3 = pivot_table3.rename_axis(index={"CAUSE.CATEGORY": "Cause Category", "CAUSE.CATEGORY.DETAIL": "Cause Category Detail"}, columns="Month")

pivot_table3.to_html('assets/images/outages_by_month', border=1, col_space=100) # Convert the pivot table to HTML and save it into my assets/images folder
pivot_table3

Month                                                    Jan  Feb  Mar  Apr  \
Cause Category                Cause Category Detail                           
equipment failure             breaker trip                 0    0    0    0   
                              cables                       0    0    0    0   
                              computer hardware            0    0    0    0   
...                                                      ...  ...  ...  ...   
system operability disruption transmission interruption    0    0    4    1   
                              uncontrolled loss            0    0    0    2   
                              voltage reduction            0    0    0    0   

Month                                                    ...  Oct  Nov  Dec  \
Cause Category                Cause Category Detail      ...                  
equipment failure             breaker trip               ...    0    0    0   
                              cables                     ...    0    0    1   
                              computer hardware          ...    0    0    0   
...                                                      ...  ...  ...  ...   
system operability disruption transmission interruption  ...    3    0    3   
                              uncontrolled loss          ...    0    0    1   
                              voltage reduction          ...    0    0    0   

Month                                                    Total  
Cause Category                Cause Category Detail             
equipment failure             breaker trip                   4  
                              cables                         1  
                              computer hardware              1  
...                                                        ...  
system operability disruption transmission interruption     17  
                              uncontrolled loss             12  
                              voltage reduction              1  

[52 rows x 13 columns]

## Step 3: Assessment of Missingness

### Missingness Dependency: `CAUSE.CATEGORY.DETAIL` (non-trivial missingness column)


In [81]:
# Checked how many missing values in each Column
missing_info = outages.isna().sum()
display(missing_info[10:16])

CAUSE.CATEGORY             0
CAUSE.CATEGORY.DETAIL    471
OUTAGE.DURATION          136
DEMAND.LOSS.MW           901
CUSTOMERS.AFFECTED       655
RES.SALES                 22
dtype: int64

In [160]:
from scipy.stats import ks_2samp

def permutation_test_nmar(df, feature_column, target_column, num_permutations=1000):
    """
    Perform a permutation test to check if the missingness of target_column depends on feature_column.

    Parameters:
    df (pd.DataFrame): The dataset.
    feature_column (str): The column we are testing for dependence.
    target_column (str): The column with missing values.
    num_permutations (int): The number of permutations to generate.

    Returns:
    observed_tvd (float): The observed TVD between missing and non-missing distributions.
    p_value (float): The p-value from the permutation test.
    tvd_null (list): List of TVD values under the null hypothesis.
    """
    # Separate the data into missing and non-missing
    missing = df[df[target_column].isna()]
    non_missing = df[df[target_column].notna()]

    # Compute observed TVD
    observed_tvd = np.abs(missing[feature_column].value_counts(normalize=True) -
                          non_missing[feature_column].value_counts(normalize=True)).sum() / 2

    # Compute TVDs for permutations
    tvd_null = []
    combined = df.copy()
    for _ in range(num_permutations):
        combined[target_column] = np.random.permutation(combined[target_column])
        perm_missing = combined[combined[target_column].isna()]
        perm_non_missing = combined[combined[target_column].notna()]
        tvd_null.append(np.abs(perm_missing[feature_column].value_counts(normalize=True) -
                               perm_non_missing[feature_column].value_counts(normalize=True)).sum() / 2)

    # Calculate p-value (proportion of null TVDs >= observed TVD)
    p_value = np.sum(np.array(tvd_null) >= observed_tvd) / num_permutations

    return observed_tvd, p_value, tvd_null


import numpy as np
import pandas as pd

# Chosen column with non-trivial missingness
column_to_analyze =  'OUTAGE.DURATION'

# Remove the column to analyze from the list of non-numeric columns
columns_to_test = ['YEAR', 'MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 
                   'CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'TOTAL.SALES', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'POPULATION', 'CUSTOMERS.AFFECTED'] 
for column in columns_to_test:
    observed_tvd, p_value, tvd_null = permutation_test_nmar(outages, column, column_to_analyze)
        # If the p-value is less than 0.05, we reject the null hypothesis
    if p_value < 0.05:
        print(f'The missingness of {column_to_analyze} depends on {column}. p-value:{p_value}, observed statistic{observed_tvd}')
    else:
        print(f'The missingness of {column_to_analyze} does not depend on {column} p-value:{p_value}, observed statistic{observed_tvd}')

The missingness of OUTAGE.DURATION depends on YEAR. p-value:0.0, observed statistic0.38740217116889675
The missingness of OUTAGE.DURATION does not depend on MONTH p-value:0.153, observed statistic0.14349520687596456
The missingness of OUTAGE.DURATION depends on U.S._STATE. p-value:0.0, observed statistic0.2762822940334933
The missingness of OUTAGE.DURATION depends on NERC.REGION. p-value:0.037, observed statistic0.13689198855507867
The missingness of OUTAGE.DURATION depends on CLIMATE.REGION. p-value:0.0, observed statistic0.27026003534460996
The missingness of OUTAGE.DURATION depends on CAUSE.CATEGORY. p-value:0.0, observed statistic0.4440902970630312
The missingness of OUTAGE.DURATION does not depend on CLIMATE.CATEGORY p-value:0.05, observed statistic0.09413335135683149
The missingness of OUTAGE.DURATION does not depend on TOTAL.SALES p-value:0.326, observed statistic0.2316017316017316
The missingness of OUTAGE.DURATION depends on CLIMATE.REGION. p-value:0.0, observed statistic0.270

#### Missingness Analysis #1: 
**Colums:** `OUTAGE.DURATION` & `MONTH`

In [173]:
import plotly.express as px
import plotly.io as pio
import pandas as pd

# Create subsets for missing and non-missing OUTAGE.DURATION
missing_duration = outages[outages['OUTAGE.DURATION'].isna()]
non_missing_duration = outages[outages['OUTAGE.DURATION'].notna()]

# Compute normalized value counts for 'MONTH'
missing_counts = missing_duration['MONTH'].value_counts(normalize=True).reset_index()
missing_counts.columns = ['MONTH', 'Missing Outage Duration']

non_missing_counts = non_missing_duration['MONTH'].value_counts(normalize=True).reset_index()
non_missing_counts.columns = ['MONTH', 'Non-Missing Outage Duration']

# Merge both counts into a single DataFrame for plotting
month_counts = pd.merge(missing_counts, non_missing_counts, on='MONTH', how='outer').fillna(0)

# Melt the dataframe to convert it into a long format
month_counts = pd.melt(month_counts, id_vars='MONTH', value_vars=['Missing Outage Duration', 'Non-Missing Outage Duration'])

# Plot using Plotly
fig = px.bar(month_counts,  
             x='MONTH',  
             y='value',  
             color='variable',  
             labels={'MONTH': 'Month',  
                     'value': 'Proportion',  
                     'variable': 'Outage Duration Status'},  
             barmode='group')  

# Customize layout
fig.update_layout(  
   title={  
      'text': 'Distribution of Outage Duration Missingness by Month',  
      'x': 0.5,  # Center the title  
      'y': 0.98,  # Add padding above the plot  
      'xanchor': 'center',  
      'yanchor': 'top',  
      'font': dict(family='Serif', size=18, color='black')},  
   legend={  
      'font': dict(family='Serif', size=12, color='black')},  
   xaxis_title_font=dict(size=14, family='Serif'),  
   xaxis_tickangle=-45,  
   yaxis_title_font=dict(size=14, family='Serif'),  
   yaxis_title='Proportion',  
   xaxis_title='Month',  
   showlegend=True,  
   width=700,  # Make the plot wider  
   height=400  
)  

# Show the plot
fig.show()

# Optionally, save the figure
fig.write_html('assets/images/OutageDuration_Month_Missingness.html', auto_open=False)



In [171]:
simulated2 = outages.copy()

observed_tvd, p_value, tvd_null = permutation_test_nmar(simulated2, 'MONTH', 'OUTAGE.DURATION')  
observed_tvd, p_value, tvd_null

null_hist = go.Histogram(
    x=tvd_null,
    nbinsx=50,
    opacity=0.5,
    name='Null distribution',
    histnorm='probability density',  # Normalize the histogram
    marker=dict(color='rgba(0, 100, 200, 0.7)')
)


# Add the observed TVD as a vertical line
observed_line = go.Scatter(
    x=[observed_tvd, observed_tvd],
    y=[0, max(np.histogram(tvd_null, bins=10)[0])],  
    mode='lines',
    name='Observed TVD',
    line=dict(color='red', width=2)
)

# Create the layout with custom x and y axis ranges and apply sans-serif font
layout = go.Layout(
    title={
        'text': 'Distribution of Outage Duration Missingness Dependence on Month',
        'x': 0.5,  # Center the title
        'y': 0.98,  # Add padding above the plot
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')},
    legend = {
        'font': dict(family='Serif', size=12, color='black') },

    xaxis_title_font=dict(size=14, family='Serif'),
    xaxis=dict(
        title='TVD Value',
        range=[min(tvd_null), 0.35]  # Extend x-axis to the right (you can change 0.4 to your desired value)
    ),
    yaxis_title_font=dict(size=14, family='Serif'),
    yaxis=dict(
        title='Density',
        range=[0, 40]  # Set y-axis range from 0 to 40
    ),
    barmode='overlay',
    showlegend=True,
    font=dict(family='Arial, sans-serif'),
    width=700,  # Make the plot wider
    height=400)



# Create the figure and plot
fig = go.Figure(data=[null_hist, observed_line], layout=layout)
fig.show()
fig.write_html('assets/images/OutageDuration_vs_Month.html', auto_open=False)


#### Missingness Analysis #2: 
**Colums:**

In [176]:
simulated2 = outages[['YEAR', 'U.S._STATE', 'TOTAL.SALES', 'MONTH', 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'CAUSE.CATEGORY.DETAIL']]


observed_tvd, p_value, tvd_null = permutation_test_nmar(simulated2, 'YEAR', 'OUTAGE.DURATION')  
observed_tvd, p_value, tvd_null

null_hist = go.Histogram(
    x=tvd_null,
    nbinsx=50,
    opacity=0.5,
    name='Null distribution',
    histnorm='probability density',  # Normalize the histogram
    marker=dict(color='rgba(0, 100, 200, 0.7)')
    )


# Add the observed TVD as a vertical line
observed_line = go.Scatter(
    x=[observed_tvd, observed_tvd],
    y=[0, max(np.histogram(tvd_null, bins=10)[0])],  
    mode='lines',
    name='Observed TVD',
    line=dict(color='red', width=2)
    )

# Create the layout with custom x and y axis ranges and apply sans-serif font
layout = go.Layout(
    title={
        'text': 'Distribution of Outage Duration Missingness Dependence on Year',
        'x': 0.5,  # Center the title
        'y': 0.98,  # Add padding above the plot
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(family='Serif', size=18, color='black')},
    legend = {
        'font': dict(family='Serif', size=12, color='black') },

    xaxis_title_font=dict(size=14, family='Serif'),
    xaxis=dict(
        title='TVD Value',
        range=[min(tvd_null), 0.45]  # Extend x-axis to the right (you can change 0.4 to your desired value)
    ),
    yaxis_title_font=dict(size=14, family='Serif'),
    yaxis=dict(
        title='Density',
        range=[0, 40]  # Set y-axis range from 0 to 40
    ),
    barmode='overlay',
    showlegend=True,
    font=dict(family='Arial, sans-serif'),
    width=700,  # Make the plot wider
    height=400   # Set font to sans-serif
    )


# Create the figure and plot
fig = go.Figure(data=[null_hist, observed_line], layout=layout)
fig.show()
fig.write_html('assets/images/OutageDuration_vs_Year.html', auto_open=False)


## Step 4: Hypothesis Testing

### Layout

**Null Hypothesis**: The number of power outages is uniformly distributed across all months of the year

**Alternative Hypothesis**: The number of outages is not uniformly distributed across all months of the year

**Test Statistic**: K2 Statistic

**Significance Level**: 

#### Justification:


In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO